In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import pickle

In [2]:
X_full = pd.read_csv('/Users/nickmac/Desktop/Dev Stuff/Kaggle/Black Friday/train.csv', index_col='User_ID')

X_test = pd.read_csv('/Users/nickmac/Desktop/Dev Stuff/Kaggle/Black Friday/test.csv', index_col='User_ID')

### Data Exploration

In [3]:
# Print the column names and their data types
print("\nTrain data information:")
print(X_full.info())


Train data information:
<class 'pandas.core.frame.DataFrame'>
Index: 550068 entries, 1000001 to 1006039
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Product_ID                  550068 non-null  object 
 1   Gender                      550068 non-null  object 
 2   Age                         550068 non-null  object 
 3   Occupation                  550068 non-null  int64  
 4   City_Category               550068 non-null  object 
 5   Stay_In_Current_City_Years  550068 non-null  object 
 6   Marital_Status              550068 non-null  int64  
 7   Product_Category_1          550068 non-null  int64  
 8   Product_Category_2          376430 non-null  float64
 9   Product_Category_3          166821 non-null  float64
 10  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 50.4+ MB
None


In [4]:
# Summary Statistics & Skewness
print("\nDescriptive Statistics (including Skewness):")
print(X_full.describe().to_markdown(numalign="left", stralign="left"))
print(X_full.skew().to_markdown(numalign="left", stralign="left"))


Descriptive Statistics (including Skewness):
|       | Occupation   | Marital_Status   | Product_Category_1   | Product_Category_2   | Product_Category_3   | Purchase   |
|:------|:-------------|:-----------------|:---------------------|:---------------------|:---------------------|:-----------|
| count | 550068       | 550068           | 550068               | 376430               | 166821               | 550068     |
| mean  | 8.07671      | 0.409653         | 5.40427              | 9.84233              | 12.6682              | 9263.97    |
| std   | 6.52266      | 0.49177          | 3.93621              | 5.08659              | 4.12534              | 5023.07    |
| min   | 0            | 0                | 1                    | 2                    | 3                    | 12         |
| 25%   | 2            | 0                | 1                    | 5                    | 9                    | 5823       |
| 50%   | 7            | 0                | 5                    | 9    

TypeError: could not convert string to float: 'P00069042'

In [5]:
# Unique Value Counts for Categorical Columns
print("\nUnique Values in Categorical Columns:")
print(X_full.select_dtypes(include='object').nunique().to_markdown(numalign="left", stralign="left"))


Unique Values in Categorical Columns:
|                            | 0    |
|:---------------------------|:-----|
| Product_ID                 | 3631 |
| Gender                     | 2    |
| Age                        | 7    |
| City_Category              | 3    |
| Stay_In_Current_City_Years | 5    |


In [6]:
# Missing Value Percentages
print("\nPercentage of Missing Values:")
print(((X_full.isnull().sum() / len(X_full)) * 100).to_markdown(numalign="left", stralign="left"))


Percentage of Missing Values:
|                            | 0       |
|:---------------------------|:--------|
| Product_ID                 | 0       |
| Gender                     | 0       |
| Age                        | 0       |
| Occupation                 | 0       |
| City_Category              | 0       |
| Stay_In_Current_City_Years | 0       |
| Marital_Status             | 0       |
| Product_Category_1         | 0       |
| Product_Category_2         | 31.5666 |
| Product_Category_3         | 69.6727 |
| Purchase                   | 0       |


### Feature Engineering

In [3]:
# Remove columns with missing target

X_full.dropna(axis=0, subset=['Purchase'], inplace=True)
y = X_full['Purchase']
X_full.drop(['Purchase'], axis=1, inplace=True)

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2,
                                                      random_state=42)

In [4]:
# Identify numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

In [5]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mean')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [9]:
# XGBoost Model with hyperparameter search space
xgb_model = XGBRegressor(random_state=42)


param_distributions = {
    'regressor__n_estimators': [50, 100, 200, 500],  # Number of trees
    'regressor__learning_rate': [0.01, 0.1, 0.2],   # Step size shrinkage
    'regressor__max_depth': [3, 5, 7],                # Maximum tree depth
    'regressor__subsample': [0.8, 0.9, 1.0],         # Fraction of samples used for fitting each tree
    'regressor__colsample_bytree': [0.8, 0.9, 1.0]   # Fraction of features used for fitting each tree
}

In [13]:
# Pipeline with Randomized Search
random_search = RandomizedSearchCV(
    estimator=Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', xgb_model)
    ]),
    param_distributions=param_distributions, 
    n_iter=10,  # Number of parameter combinations to try
    scoring='neg_mean_absolute_error',
    cv=5,  #10 x 5 would result in 50 fits -> not sure if CV necessary here
    verbose=1,
)

random_search.fit(X_train, y_train)

print("Best parameters found by random search:", random_search.best_params_)
print("Best MAE found by random search:", -random_search.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


KeyboardInterrupt: 

In [ ]:
# Grid Search (Refine around best parameters from random search)
param_grid = {
    'regressor__n_estimators': [random_search.best_params_['regressor__n_estimators'] - 25, 
                                random_search.best_params_['regressor__n_estimators'], 
                                random_search.best_params_['regressor__n_estimators'] + 25],
    'regressor__learning_rate': [random_search.best_params_['regressor__learning_rate'] - 0.05, 
                                 random_search.best_params_['regressor__learning_rate'], 
                                 random_search.best_params_['regressor__learning_rate'] + 0.05],
    'regressor__max_depth': [random_search.best_params_['regressor__max_depth'] - 1,
                            random_search.best_params_['regressor__max_depth'],
                            random_search.best_params_['regressor__max_depth'] + 1]
}

grid_search = GridSearchCV(estimator=random_search.best_estimator_, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best model from grid search and make predictions
best_model = grid_search.best_estimator_

print("Best parameters found by grid search:", grid_search.best_params_)
print("Best MAE found by grid search:", -grid_search.best_score_)

In [ ]:
preds = best_model.predict(X_valid)

# Calculate MAE on the validation set
mae_valid = mean_absolute_error(y_valid, preds)
print(f"Mean Absolute Error (Validation Set): {mae_valid}")

In [ ]:
# save the model to disk
filename = 'black_friday_final_model.sav'
pickle.dump(best_model, open(filename, 'wb'))

----------

In [ ]:
# # Convert processed data back to DataFrames
# X_train_processed = pd.DataFrame(X_train_processed, columns=preprocessor.get_feature_names_out())
# X_valid_processed = pd.DataFrame(X_valid_processed, columns=preprocessor.get_feature_names_out())

In [ ]:
# # Fit and transform training data
# X_train_processed = preprocessor.fit_transform(X_train)

# # Transform validation data (using the same preprocessor)
# X_valid_processed = preprocessor.transform(X_valid)

In [ ]:
# # XGBoost Model
# xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.1, random_state=42)

# # Create the full pipeline
# full_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('regressor', xgb_model) 
# ])

In [ ]:
# # Fit the pipeline (includes preprocessing and model training)
# full_pipeline.fit(X_train, y_train, regressor__early_stopping_rounds=5, regressor__eval_set=[(X_valid, y_valid)], regressor__verbose=False)


# # Get predictions on the validation set
# preds = full_pipeline.predict(X_valid)

In [ ]:
# # Separate numerical and categorical columns
# X_train_num = X_train.select_dtypes(exclude='object')
# X_train_cat = X_train.select_dtypes(include='object')

# X_valid_num = X_valid.select_dtypes(exclude='object')
# X_valid_cat = X_valid.select_dtypes(include='object')

In [ ]:
# # 1. Numerical Imputation (Mean)
# num_imputer = SimpleImputer(strategy='mean')  
# imputed_X_train_num = pd.DataFrame(num_imputer.fit_transform(X_train_num))
# imputed_X_valid_num = pd.DataFrame(num_imputer.transform(X_valid_num))

# # Reassign column names after imputation
# imputed_X_train_num.columns = X_train_num.columns
# imputed_X_valid_num.columns = X_valid_num.columns

In [ ]:
# # 2. Categorical Imputation (Most Frequent)
# cat_imputer = SimpleImputer(strategy='most_frequent')
# imputed_X_train_cat = pd.DataFrame(cat_imputer.fit_transform(X_train_cat))
# imputed_X_valid_cat = pd.DataFrame(cat_imputer.transform(X_valid_cat))

# # Reassign column names after imputation
# imputed_X_train_cat.columns = X_train_cat.columns
# imputed_X_valid_cat.columns = X_valid_cat.columns

In [ ]:
# # One-Hot Encoding
# OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(imputed_X_train_cat))
# OH_cols_valid = pd.DataFrame(OH_encoder.transform(imputed_X_valid_cat))

# # Restore index after encoding
# OH_cols_train.index = imputed_X_train_cat.index
# OH_cols_valid.index = imputed_X_valid_cat.index

# # Ensure all columns have string type
# OH_cols_train.columns = OH_cols_train.columns.astype(str)
# OH_cols_valid.columns = OH_cols_valid.columns.astype(str)

In [ ]:
# # Combine numerical and OH encoded data
# final_X_train = pd.concat([imputed_X_train_num, OH_cols_train], axis=1)
# final_X_valid = pd.concat([imputed_X_valid_num, OH_cols_valid], axis=1)

To Do

- Get all numerical and categorical variables
- Impute numerical variables and replace categorical with most frequent strategy
- You also need to process the test data